In [7]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import os
from multiprocessing import Process

os.chdir("c:\\Users\\swart\\Desktop\\secure-mpc-main\\basic_networking")
from client import Client
from server import Server
os.chdir("c:\\Users\\swart\\Desktop\\secure-mpc-main\\ecgPreprocessedData")
from Data.SignalDataset import SignalDataset
from PickleFileUtils import read_in_pickle_file

In [8]:
complete_data_set = read_in_pickle_file("data\\CompleteDataSetFeatures.pickle")
original_training_data_set = read_in_pickle_file("data\\OriginalTrainingDataSetFeatures.pickle")
original_validation_data_set = read_in_pickle_file("data\\OriginalValidationDataSetFeatures.pickle")

print(len(complete_data_set))
print(len(original_training_data_set))
print(len(original_validation_data_set))

6029
5834
195


In [9]:
# get labels
labels = complete_data_set.labels

# get data
data = complete_data_set.data

data,labels = shuffle(data, labels)
X_class = np.array(data)
y_class = np.array(labels)

In [4]:
alice = Client()
server = Server()

def test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, client: Client, server: Server, rand_range):
	server.set_features_labels(X_train, y_train)
	y_pred = []
	for test_row in X_test:
		prediction = client.predict_classification(test_row, num_neighbors, server, rand_range)
		y_pred.append(prediction)

	f1_binary = f1_score(y_test, y_pred, average="binary")
	accuracy = accuracy_score(y_test, y_pred)

	# print("ypred:", y_pred)
	# print("ytest:", y_test)

	return f1_binary, accuracy



In [9]:
#testing
def test(rand_range):
    numFolds = 10
    num_neighbors = 9
    stratifiedKFold = StratifiedKFold(
        n_splits=numFolds, shuffle=True, random_state=86
    )

    count = 1
    avgF1 = 0
    avgAcc = 0

    X = X_class
    y = y_class

    server = Server()
    alice = Client()

    for train_index, test_index in stratifiedKFold.split(X, y):
        print("On fold ", count)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        f1, acc = test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, alice, server, rand_range)
        avgF1 += f1
        avgAcc += acc

        print(f"\tFold {count}:: Accuracy: {acc}, F1 score: {f1}, randRange: {rand_range}")

        count += 1

    avgF1 = avgF1/numFolds
    avgAcc = avgAcc/numFolds

    # return_vals[result_place] = [avgAcc, avgF1, rand_range]

    return avgAcc, avgF1



In [10]:
def test_threaded(x_val, return_vals, result_place):
    avgAcc, avgF1 = test(x_val)
    return_vals[result_place] = [avgAcc, avgF1, x_val]

# test with multiple lagrange max x-points
from threading import Thread


x_max_list = [100,1000,10000,100000,1000000]

print("Testing 2017 AF Dataset - SMPC")


# get thread list
thread_list = []
thread_results = [[None]*5]*5
for i in range(len(x_max_list)):
    thread = Thread(target=test_threaded, args=(x_max_list[i],thread_results[i],i))
    thread_list.append(thread)
    thread.start()

for thread in thread_list:
    thread.join()

thread_results = thread_results[0]

print(thread_results)

Testing 2017 AF Dataset - SMPC
On fold  1
On fold  1
On fold  1
On fold  1
On fold  1


c:\Users\swart\Desktop\secure-mpc-main\basic_networking\client.py:50: RuntimeWarning: invalid value encountered in sqrt
  distance = abs(np.sqrt(s_0))


In [1]:
smpc_af_results = [[0.9527288805143715, 0.8129754369188585, 100], 
                   [0.9527288805143715, 0.8129754369188585, 1000], 
                   [0.9527288805143715, 0.8129754369188585, 10000], 
                   [0.9527288805143715, 0.8129754369188585, 100000], 
                   [0.9525633185126416, 0.8128837632950312, 1000000]]

for i in smpc_af_results:
    print(f"Accuracy: {i[0]}, F1: {i[1]}, Max X for Lagrange: {i[2]}")

Accuracy: 0.9527288805143715, F1: 0.8129754369188585, Max X for Lagrange: 100
Accuracy: 0.9527288805143715, F1: 0.8129754369188585, Max X for Lagrange: 1000
Accuracy: 0.9527288805143715, F1: 0.8129754369188585, Max X for Lagrange: 10000
Accuracy: 0.9527288805143715, F1: 0.8129754369188585, Max X for Lagrange: 100000
Accuracy: 0.9525633185126416, F1: 0.8128837632950312, Max X for Lagrange: 1000000


In [11]:
print(sum(y_class))
print(len(y_class))

805
6029
